In [6]:
import itertools
import random
import simpy
import  Agents

## Utils

### Class

In [7]:
class My_container(simpy.Container):
    def __init__(self, env, name: str, capacity: int | float = ..., init: int | float = 0):
        super().__init__(env, capacity, init)
        self.name = name


class My_resource(simpy.Resource):
    def __init__(self, env, name: str, container: My_container=None, capacity: int = 1):
        super().__init__(env, capacity)
        self.name = name
        self.container = container

class Resources_set:
    def __init__(self, env, count_resources, name, container: My_container=None):
        self.env = env
        self.resource_name = name
        self.resources = [My_resource(env, f'{name}_{i}', container, capacity=1) for i in range(count_resources)]

    def request_resource(self):
        # Genera un evento para cada recurso
        for resource in self.resources:
            yield resource.request()  # Genera un evento de solicitud

        # Genera un evento que nunca se completa (para el "break")
        yield self.env.event()
        

    def release_resource(self, resource):
        resource.release(resource)


### Functions

In [8]:
def rule_of_three(numerator, denominator, factor):
    return (numerator/denominator)*factor

## Definiendo parámetros

In [9]:
""" SIMULATION """
RANDOM_SEED = 42
T_INTER = [30, 300]            # intervalo entre la llegada de los turistas
SIM_TIME = 1000                # tiempo total de la simulación

""" ROOM """
ROOM_CLEANING_SIZE = 200       # máximo nive de limpieza de una habitación 
THRESHOLD_CLEAN = 80           # mínimo de limpieza/confort (% del total)

""" POOL """
POOL_CLEANING_SIZE = 100       # máximo nive de limpieza de una habitación 
THRESHOLD_CLEAN = 50           # mínimo de limpieza/confort (% del total)

""" TOURIST """
TOURIST_ENERGY_SIZE = 100        # nivel máximo de descanso
TOURIST_ENERGY_LEVEL = [20, 50]  # nivel inicial de energía de los turistas (menor_energía => más_sueño)
TOURIST_HUNGER_SIZE = 100        # nivel máximo de llenura por comida
TOURIST_HUNGER_LEVEL = [20, 50]  # nivel inicial de hambre de los turistas (menor_nivel => más_hambre)
TOURIST_FUN_SIZE = 100           # nivel máximo de diversión
TOURIST_FUN_LEVEL = [20, 50]     # nivel inicial de diversión de los turistas
TOURIST_COMFORT_SIZE = 100       # nivel máximo de confort
TOURIST_COMFORT_LEVEL = [20, 50] # nivel inicial de confort de los turistas (menor_energía => más_sueño)

TOURIST_ROOMS = {}

LEN_OF_STAY = [100, 200]
SLEEP_SPEED = 2                  # velocidad de recuperar energía (u / second)

""" WORKER """
HOUSEMAID_TIME = 50           # tiempo que tarda la mucama en limpiar la habitación (segundos)


""" MONEY """
PRICES = {}
PRICES['room'] = 50
PRICES['pool'] = 10
PRICES['buffet'] = 20
PRICES['bar'] = 30

SALARIES = {}
SALARIES['housemaid'] = 5

SALARIES_AMOUNT = {}             # salario cobrado por cada trabajador
SALARIES_AMOUNT['housemaid'] = 0

AMOUNT = {}                    # pago acumulado por Los turistas en cada servicio
AMOUNT['room'] = 0
AMOUNT['pool'] = 0
AMOUNT['buffet'] = 0
AMOUNT['bar'] = 0
#AMOUNT['room'] = (0, 0, 0)    # (pago_del_turista, del_1ro_lo_pagado_al_trabajador, cant_turist)

### Red social

In [5]:
SATISFACTION = {} # ['tourist_name'] = {['necesity'] = level_when_leaves_the_hotel}

# Agents
## Tourist

**Creencias (Beliefs):**
El nivel de cada una de las necesidades al llegar al hotel.


**Deseos (Desires):**
Satisfacer todas las necesidades


**Intenciones (Intentions):**



**Variables y Deducciones:**


In [6]:
#from collections import deque

def tourist(name, env, hotel, len_of_stay):
    """Función que representa un turista con comportamiento BDI."""
    beliefs = {
        'energy_level': random.randint(*TOURIST_ENERGY_LEVEL),
        'hunger_level': random.randint(*TOURIST_HUNGER_LEVEL),
        'fun_level': random.randint(*TOURIST_FUN_LEVEL),
        'comfort_level': random.randint(*TOURIST_COMFORT_LEVEL),
        'has_room': False,
        'room_cleanliness': None,
    }
    desires = {
        'want_energy': True,
        'want_food': False,
        'want_fun': False,
        'want_comfort': False,
        'want_room': True,
    }
    intentions = []
    # global current_room
    TOURIST_ROOMS[name] = None # Para guardar la habitación actual
    #print(f'Believes actuales del {name}: {beliefs}\n Tiempo de estancia: {len_of_stay}')
    

    def update_beliefs():
        # Actualiza las creencias según las acciones realizadas
        #global current_room
        if beliefs['has_room']:
            beliefs['room_cleanliness'] = TOURIST_ROOMS[name].container.level
        if beliefs['energy_level'] >= TOURIST_ENERGY_SIZE:
            desires['want_energy'] = False
        if beliefs['hunger_level'] >= TOURIST_HUNGER_SIZE:
            desires['want_food'] = False
        if beliefs['fun_level'] >= TOURIST_FUN_SIZE:
            desires['want_fun'] = False
        if beliefs['comfort_level'] >= TOURIST_COMFORT_SIZE:
            desires['want_comfort'] = False
        #print(f'beliefs actualizados del {name}: {beliefs}')

    def choose_intention():
        update_beliefs()
        if desires['want_room'] and not beliefs['has_room']:
            intentions.append(reserve_room)
            return
        if desires['want_energy'] and beliefs['has_room'] and beliefs['room_cleanliness'] >= (TOURIST_ENERGY_SIZE - beliefs['energy_level']):
            intentions.append(rest_in_room)
            return
        if desires['want_food']:
            intentions.append(go_to_buffet)
            #return
        if desires['want_fun'] and not beliefs['has_room']:
            intentions.append(go_to_pool)
            #return
       # print(f'Intentions of {name}: {intentions}')

    def execute_intention():
        if intentions:
            intention = intentions.pop()
            #print(f'{name} executing intention {intention}')
            env.process(intention())

    def reserve_room():
        for res in hotel['energy']:
            if res.resource_name == 'room':
                for room in res.resources:
                    if room.count == 0:
                        with room.request() as request_room:
                            yield request_room
                            beliefs['has_room'] = True
                            desires['want_room'] = False                            
                            TOURIST_ROOMS[name] = room
                            AMOUNT['room'] += PRICES['room']
                            print(f'El turista {name} accedió a la habitación {room.name}.')
                            yield env.timeout(len_of_stay)
                            break  # Salir del bucle una vez que se reserve una habitación
                break


    def rest_in_room():
        if beliefs['has_room'] and desires['want_energy']:
            energy_required = TOURIST_ENERGY_SIZE - beliefs['energy_level']
            if energy_required <= 0: return
         
            #global current_room
            # print(f'limpieza de la habitación: {TOURIST_ROOMS[name].container.level}')
            # print(f'energy required: {energy_required}')
            if TOURIST_ROOMS[name].container.level >= energy_required:
                yield TOURIST_ROOMS[name].container.get(energy_required)
                yield env.timeout(SLEEP_SPEED)
                beliefs['energy_level'] += energy_required
                desires['want_energy'] = False
                
                print(f'El turista {name} ha descansado en la habitación. Energía: {beliefs['energy_level']}.')
            else:
                print(f'El turista {name} no pudo descansar. No tiene habitación para dormir.')
                #print(f'La habitación {TOURIST_ROOMS[name]} está ocupada por {TOURIST_ROOMS[name].count} turistas')

    def go_to_pool():
        # Lógica para ir a la piscina (similar a reservar una habitación)
        pass

    def go_to_buffet():
        # Lógica para ir al buffet (similar a reservar una habitación)
        pass
    now = env.now
    print(f'{name} start at {now}')

    while env.now < now + len_of_stay:
        choose_intention()
        execute_intention()
        yield env.timeout(random.randint(1, 3))  # Tiempo de espera
    print(f'{name} left the hotel at {env.now} with ')
    if name not in SATISFACTION:
      SATISFACTION[name] = {}        
    SATISFACTION[name]['energy'] = beliefs['energy_level']
    SATISFACTION[name]['hunger'] = beliefs['hunger_level']
    SATISFACTION[name]['fun'] = beliefs['fun_level']
    SATISFACTION[name]['comfort'] = beliefs['comfort_level']
    print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
    print(f'Actual level of clean of {TOURIST_ROOMS[name].name}: {TOURIST_ROOMS[name].container.level}')
    TOURIST_ROOMS[name] = None






# def tourist(name, env, hotel): # most be include 'age'
   
#     tourist_energy_level = random.randint(*TOURIST_ENERGY_LEVEL)
#     tourist_hunger_level = random.randint(*TOURIST_HUNGER_LEVEL)
#     tourist_fun_level = random.randint(*TOURIST_FUN_LEVEL)
#     tourist_comfort_level = random.randint(*TOURIST_COMFORT_LEVEL)

#     # Beliefs: Estado actual del turista
#     beliefs = {
#         'energy': tourist_energy_level,
#         'hunger': tourist_hunger_level,
#         'fun': tourist_fun_level,
#         'comfort': tourist_comfort_level
#     }

#     # Desires: Necesidades del turista
#     desires = [
#         ('energy', TOURIST_ENERGY_SIZE - tourist_energy_level),
#         ('hunger', TOURIST_HUNGER_SIZE - tourist_hunger_level),
#         ('fun', TOURIST_FUN_SIZE - tourist_fun_level),
#         ('comfort', TOURIST_COMFORT_SIZE - tourist_comfort_level)
#     ]

#     # Intentions: Acciones basadas en creencias y deseos
#     intentions = []
#     for desire in desires:
#         resource_type, _ = desire
#         for resource in hotel[resource_type]:
#             intentions.append((resource_type, resource))

#     #print(f'{env.now:6.1f} s: {name} arrived at room')

#     for room in hotel.rooms.resources:
#         if room.count == 0:
#             # Solicita un recurso 
#             with room.request() as request_room:
#                 yield request_room
#                 print(f"El turista {name} accedió a la habitación {room.name} con {tourist_energy_level} de energy.")

#                 # Get the required amount of energy
#                 energy_required = TOURIST_ENERGY_SIZE - tourist_energy_level

#                 if room.container.level < energy_required:
#                     print(f'El turista {name} no pudo descansar. La habitación {room.name} estaba DEMASIADO sucia. Se ha ido SIN PAGAR')
#                     print(f'Level of clean: {room.container.level}')
#                     print(f'Energy required: {energy_required}')

#                     if name not in SATISFACTION:
#                         SATISFACTION[name] = {}        
#                     SATISFACTION[name]['energy'] = tourist_energy_level
#                     print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
#                     print(f'Actual level of clean of {room.name}: {room.container.level}')
#                     print(f'{name} left the {room.name} with {tourist_energy_level} (not full energy)')
#                     break

#                 else:
#                     AMOUNT['room'] += PRICES['room']
#                     yield room.container.get(energy_required)

#                     # The "actual" recovering of energy process takes some time
#                     yield env.timeout(energy_required / SLEEP_SPEED)

#                     if name not in SATISFACTION:
#                         SATISFACTION[name] = {}        
#                     SATISFACTION[name]['energy'] = tourist_energy_level + energy_required
#                     #print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
#                     print(f'Actual level of clean of {room.name}: {room.container.level}')
#                     print(f'{name} left the {room.name} with {tourist_energy_level + energy_required} (full energy)')

#                     break




    


## Workers

In [7]:
def housemaid(env, room):
    """Arrives at the room and finish of clean it after a certain delay."""
    #add si no tiene el salario completo no rellena al máximo el nivel de limpieza
    yield env.timeout(2)

    with room.request() as rq:
        yield rq
        print(f'{env.now:6.1f} s: Housemaid is cleaning the {room.container.name} of the {room.name}...')
        bed = room.container
        amount = bed.capacity - bed.level
        print(f'{env.now:6.1f} s: Level before clean the {room.name}: {bed.level}')
        bed.put(amount)
        yield env.timeout(HOUSEMAID_TIME)

    print(
        f'{env.now:6.1f} s: Housemaid finished and the room is clean'
    )
    print(f'Level after clean {room.name}: {bed.level}')

def interviewer(env):
    pass

def receptionist(env):
    pass

def bartender(env):
    pass

def pool_cleaner(env):
    pass



## Manager

In [8]:
def manager(env, rooms):
    """Periodically check the clean level of the room and call the housemaid
       if the level falls below a threshold."""
    while True:
        # DO SOMETHING FOR VERIFY THE EVOLUTION OF THE AMOUNT%%%%%%%%%%%%%%%%%%%%$$$$$$$$$$$$$$$$###################
        # 

        
        # actual_amount = AMOUNT
        # yield env.timeout(30)
        # diff = DeepDiff(actual_amount, AMOUNT)
        # print(f'Actualization of evolution of amount: {diff}')

        for room in rooms:

            if room.container.level / room.container.capacity * 100 < THRESHOLD_CLEAN:
                # We need to call the housemaid now!
                
                # Wait for the housemaid to clean the room
                yield env.process(housemaid(env, room))

                housemaid_salary = SALARIES['housemaid']

                if AMOUNT['room'] >= housemaid_salary:
                    AMOUNT['room'] -= housemaid_salary
                    SALARIES_AMOUNT['housemaid'] += housemaid_salary
                    #print(f'{env.now:6.1f} s: The housemaid charaged ${housemaid_salary}. The amount salary of housemaid is {SALARIES_AMOUNT['housemaid']}')

                # else: (algo con el rendimiento de la housemaid al limpiar la habitación)
        
        yield env.timeout(5)  # Check every 5 seconds

        # añadir análisis con las ganancias

In [9]:
def tourist_generator(env, hotel):
    """Generate new tourists that arrive at the hotel."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(tourist(f'Tourist {i}', env, hotel, random.randint(*LEN_OF_STAY)))

# Simulation

In [111]:
# Setup and start the simulation
import simpy.resources
import simpy.resources.resource
import hotel
print('Hotel is open')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()  # hotel

#hotel = {'energy': [], 'hunger': [], 'fun': [], 'comfort': []}

#rooms = Resources_set(env, 10, 'room', My_container(env, 'bed', ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE))
#pool = Resources_set(env, 1, 'pool', My_container(env, 'bed', ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE))

#hotel['energy'] = [rooms, pool]

pool = hotel.Service(simpy.Resource(env, capacity=11), 'pool', 'fun', hotel.Utility('pool_utl', simpy.Container(env, ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE)))

env.process(manager(env, rooms.resources))
env.process(tourist_generator(env, hotel))

# Execute!
env.run(until=SIM_TIME)


Hotel is open
Tourist 0 start at 87
El turista Tourist 0 accedió a la habitación room_0.
El turista Tourist 0 ha descansado en la habitación. Energía: 100.
Tourist 0 left the hotel at 191 with 
Actual level of satisfaction: {'Tourist 0': {'energy': 100, 'hunger': 27, 'fun': 24, 'comfort': 43}, 'Tourist 1': {'energy': 100, 'hunger': 26, 'fun': 42, 'comfort': 22}, 'Tourist 2': {'energy': 159, 'hunger': 43, 'fun': 23, 'comfort': 41}, 'Tourist 3': {'energy': 100, 'hunger': 40, 'fun': 36, 'comfort': 39}, 'Tourist 4': {'energy': 178, 'hunger': 39, 'fun': 22, 'comfort': 41}, 'Tourist 5': {'energy': 159, 'hunger': 46, 'fun': 37, 'comfort': 29}, 'Tourist 6': {'energy': 100, 'hunger': 28, 'fun': 48, 'comfort': 49}}
 Actual amount: {'room': 33885, 'pool': 0, 'buffet': 0, 'bar': 0}
Actual level of clean of room_0: 127
Tourist 1 start at 257
El turista Tourist 1 accedió a la habitación room_0.
El turista Tourist 1 ha descansado en la habitación. Energía: 100.
Tourist 1 left the hotel at 436 with 
A